# Hypothesis Testing, A/B Testing
---

Tonight we'll be conducting our own A/B test and extracting useful business insights!

In [ ]:
from datascience import *
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

You've just been hired as the Chief Data Scientist for CALPIRG.  You're being tasked with helping the UX (user experience) team tweak your website to try to get more people to pledge to Save the Bees™.  Here are the two strategies the team has come up with:
1. Have a pledge button and call-to-action on just the main page of the site
2. Have a popup with the pledge button and call-to-action whenever you visit the site

As a data scientist, it is your job to see which method is more effective at driving pledges (we'll simply call them "clicks").  It is also your job to create a compelling argument to convince CALPIRG to act in accordance to your findings.

Ready?  Let's begin.

# Step 0: Why do we do hypothesis testing?
---

Simply comparing the average of the two groups has a lot of room for error.

Have you ever heard of a difference being "*statistically significant*"?  We don't just want to see if there's a difference—we want to know if that difference is caused by something, or if it's even a significant difference in the first place!

We're able to determine if something is significant or not by hypothesis testing.

Hypothesis testing helps with the following:
* Figuring out the differences between two measurements when we're looking at a **sample** instead of the population
* Figuring out if a difference could have happened by random chance
* Establishing **causation** if we conducted an experiment

In [ ]:
# That last point is very important

# Step 1: Choosing our hypotheses
---

Recall, when conducting a hypothesis test we generate two hypotheses:

### The Null Hypothesis (H<sub>0</sub>):
States that nothing really changed.  Technically, it states that both of the samples came from the same population.

### The Alternative Hypothesis (H<sub>1</sub>):
States that there is a difference between the two distributions from which the samples were taken.

In [ ]:
# Alright, let's create our two hypotheses together

```
Null: ...
Alt:  ...
```

In [ ]:
# It's important to be specific when working with statistics.

In [ ]:
# Let's create two more hypotheses:

```
Let c := the average click rate of visitors before the experiment (theoretically the same as control)

H_0: ...
H_1: ...

or

H_0: ...
H_1: ...
```

# Step 2: Setting up the experiment
---

Now we need to figure out how to test this (and how to measure the results).

Since we are scientists, after all, it is important that we follow some rules when conducting experiements:
- The experiment must be controlled
- The experiment must have a treatment and control group
- The experiment must not be harmful
- The experiment must produce results that can evaluate a hypothesis
- The results must be reproducible (the experiment must be able to be repeated)

Well, I hope we're not harming anyone by showing them a few popups.  As for fulfilling the rest of the requirements, what do you think is a good way to create our experiment?

```
...
```

Our experiment is a Random Controlled Trial (RCT), as opposed to an Observational Study.  What does this mean we can conclude from the results of the experiment?

```
...
```

# Step 3: Execute the experiment
---

Now we implement this experiment in the code for the website.  We need to make sure to record the results!

We received 1000 visitors over the last week, and kept track of which group they were in and whether they pledged.

`0 = Did not pledge; 1 = Pledged`

In [ ]:
data = Table.read_table("popup-test.csv")

In [ ]:
data

In [ ]:
# Is this the population?

```
...
```

# Step 4: Test the null hypothesis
---

Let's establish something thing before we go any further:
- What is the statistic we are trying to test for?

In [ ]:
# What are we looking at between treatment and control?

```
...
Let's be specific: ...
```

In [ ]:
# Let's find that difference from our sample above
num_clicks = ...
num_clicks

In [ ]:
group_counts = ...
group_counts

In [ ]:
click_rates = Table().with_columns(
    "Group", ["Control", "Treatment"],
    "Rate", num_clicks.column("Click sum") / group_counts.column("count")
)
click_rates

In [ ]:
rates = ...
observed_diff = ...
observed_diff

In [ ]:
# Does that seem like a big difference?

```
...
```

Now we are trying to figure out how likely it is to observe that distance or greater.

In [ ]:
# Why "that distance or greater"?  Why not "that distance"?

```
...
```

### A bit more detail into testing the null hypothesis

What is the null hypothesis?

> The popup has no effect.  The click rate for visitors who get the popup and for visitors who don't come **from the same population**.

Recall, when we run a hypothesis test we are **trying to prove the null hypothesis**.  How would we do that?

```
...
```

### Permutation test
All we have is a sample.  We have no other knowledge of our population.

So... how are we going to try and prove the null hypothesis?

```
...
```

In [ ]:
# Let's conduct the permutation test
trials = 1000
differences = ...

for ...:
    # We choose one of the columns, either the Group or the Click, to shuffle
    shuffled_groups = data...
    
    # Then we match it back up with our other column
    shuffled_data = Table().with_columns(
        "Group", shuffled_groups,
        "Click", data.column("Click")
    )
    
    # Now, we see what differences we get after shuffling
    click_rates = (
        shuffled_data.group("Group", sum).column("Click sum")
        / shuffled_data.group("Group").column("count")
    )
    
    difference = ...
    
    differences = ...

In [ ]:
# Let's visualize the distribution of differences
Table().with_column("Difference", differences)...

# Where does our observed difference land?
plt.axvline(observed_diff, c='r')

In [ ]:
# Wow, that's more different than 6% would lead you to believe!

In [ ]:
# What's the chance of a difference geq the observed difference occurring?
...

### Another type of test
Let's say that we know the average click rate for visitors is 0.4.  Then, we can test the null hypothesis by simulation as opposed to permutation.

In [ ]:
trials = 1000
visitors = 1000
differences = []

for i in range(trials):
    d = {"Control": {"visitors": 0, "clicks": 0}, "Treatment": {"visitors": 0, "clicks": 0}}
    
    for visitor in range(visitors):
        # 50% chance of being in treatment
        ...

        # 40% chance of clicking
        ...
        
        d[group]["visitors"] += 1
        d[group]["clicks"] += click
        
    rate_control = d["Control"]["clicks"] / d["Control"]["visitors"]
    rate_treatment = d["Treatment"]["clicks"] / d["Treatment"]["visitors"]
    difference = rate_control - rate_treatment
    
    differences.append(difference)

In [ ]:
# Visualize again
Table().with_column("Difference", differences).hist()
plt.axvline(observed_diff, c='r')

In [ ]:
...

# Step 5: Interpret the results
---

In order to interpret the results, and figure out whether or not the observed difference was statistically significant, we need to set a significance level, **<code>&alpha;</code>**.

The significance level is defined as the probability of making a "Type I Error" in which we reject the null hypothesis when it is actually true.  Meaning, if we set $\alpha = 0.05$ then we have 5% chance of mistakenly rejecting the null hypothesis.


From our experiment, we calculate a **<code>p-value</code>**.

The p-value is the defined as the probability of observing a difference greater than or equal to the observed difference if the null hypothesis were to be true.

In [ ]:
# What is the p-value of our experiment above?
# (based on the permutation test)

In [ ]:
...

If the p-value is less than or equal to the significance level, then we reject the null hypothesis and support the alternative hypothesis.

In [ ]:
# Since this is an experiment, what does supporting the alternative hypothesis imply?

```
...
```

In [ ]:
# If the p-value is greater than the significance level, then we...

```
If p > 𝛼, then we ...
```

So, now it's time to prepare our presentation to CALPIRG:

---

In order to get more people to pledge while visiting the website, we should:
...

We concluded this from
...
in which we assigned
...
We collected
...
and saw
...

We proposed two hypotheses:
...
After running a
...
we discovered that the observed difference
...

So, we
...
because our p-value
...
And, therefore
...

In conclusion,
...

# Have a great Thanksgiving!